In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikisql", trust_remote_code=True)


In [ ]:
# Check the structure of the dataset
print(dataset)

# Check a sample from the training data
print(dataset['train'][15])


In [ ]:
def preprocess_data(example):
    # Directly use the 'human_readable' query as the target
    sql_query = example['sql']['human_readable']
    
    return {
        "input_text": example["question"],  # The natural language question
        "target_text": sql_query  # The human-readable SQL query
    }

# Apply preprocessing to the train and validation sets
train_data = dataset["train"].map(preprocess_data, remove_columns=["question", "sql"])
val_data = dataset["validation"].map(preprocess_data, remove_columns=["question", "sql"])

# Check a preprocessed sample
print(train_data[0])


In [ ]:
from transformers import BartTokenizer

# Load the tokenizer for BART
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Tokenization function
def tokenize_data(example):
    # Tokenize both input and target texts
    inputs = tokenizer(example['input_text'], padding='max_length', truncation=True, max_length=512)
    targets = tokenizer(example['target_text'], padding='max_length', truncation=True, max_length=64)
    
    # Return the tokenized version
    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': targets['input_ids']
    }

# Tokenize the train and validation datasets
train_data = train_data.map(tokenize_data, remove_columns=['input_text', 'target_text'])
val_data = val_data.map(tokenize_data, remove_columns=['input_text', 'target_text'])

# Check a tokenized sample
print(train_data[0])


In [ ]:
print(train_data[0])  # Check a sample after tokenization


In [ ]:
import os

# Disable Weights and Biases logging
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

# Load the BART model for conditional generation
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    load_best_model_at_end=True,
    report_to="none",
    fp16=True,
    dataloader_num_workers=8,  # Increase number of workers
)


# Initialize the Trainer
trainer = Trainer(
    model=model,                        # The model to train
    args=training_args,                 # Training arguments
    train_dataset=train_data,           # The training dataset
    eval_dataset=val_data,              # The validation dataset
)

# Train the model
trainer.train()


In [ ]:
import torch

# Clear cache in PyTorch
torch.cuda.empty_cache()



In [ ]:
# Save the trained model and tokenizer
model.save_pretrained('/kaggle/working/final_model')
tokenizer.save_pretrained('/kaggle/working/final_model')


In [ ]:
# Evaluate the model using the Trainer
trainer.evaluate()


In [ ]:
import shutil

# Create a ZIP of the final model folder (or any folder you want to download)
shutil.make_archive('/kaggle/working/final_model', 'zip', '/kaggle/working/final_model')


In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer
import sqlite3

# Define the tables in a string format (for model input)
employees_table = """
Employees table:
ID | Name   | Department  | Salary
1  | John   | HR          | 50000
2  | Alice  | Engineering | 70000
3  | Bob    | Engineering | 75000
4  | Clara  | HR          | 55000
5  | David  | Engineering | 80000
"""

departments_table = """
Departments table:
ID | Department_Name | Location
1  | HR              | New York
2  | Engineering     | San Francisco
3  | Marketing       | Chicago
"""

# Define the question
question = "What is the average salary of employees in each department located in New York?"

# Combine the tables and question into a single input string
input_text = employees_table + "\n" + departments_table + "\nQuestion: " + question

# Load the trained model and tokenizer (replace with your model path)
model = BartForConditionalGeneration.from_pretrained('/kaggle/input/bart/transformers/default/1')  # Replace with your model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Tokenize the input
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

# Generate SQL query from the model
output = model.generate(inputs['input_ids'], max_length=100, num_beams=5, early_stopping=True)

# Decode the generated SQL query
generated_sql = tokenizer.decode(output[0], skip_special_tokens=True)


# Display the generated SQL query
print("Generated SQL Query:", generated_sql)

# Set up the SQLite in-memory database and insert data into both tables
conn = sqlite3.connect(':memory:')  # Using in-memory database for testing
cursor = conn.cursor()

# Create Employees table
cursor.execute('''CREATE TABLE Employees (
                    ID INTEGER PRIMARY KEY,
                    Name TEXT,
                    Department TEXT,
                    Salary INTEGER)''')

# Create Departments table
cursor.execute('''CREATE TABLE Departments (
                    ID INTEGER PRIMARY KEY,
                    Department_Name TEXT,
                    Location TEXT)''')

# Insert data into Employees table
employees_data = [
    (1, 'John', 'HR', 50000),
    (2, 'Alice', 'Engineering', 70000),
    (3, 'Bob', 'Engineering', 75000),
    (4, 'Clara', 'HR', 55000),
    (5, 'David', 'Engineering', 80000)
]
cursor.executemany('INSERT INTO Employees VALUES (?, ?, ?, ?)', employees_data)

# Insert data into Departments table
departments_data = [
    (1, 'HR', 'New York'),
    (2, 'Engineering', 'San Francisco'),
    (3, 'Marketing', 'Chicago')
]
cursor.executemany('INSERT INTO Departments VALUES (?, ?, ?)', departments_data)

conn.commit()

# Execute the corrected SQL query
cursor.execute(generated_sql)

# Fetch the result (average salary of employees in departments located in New York)
result = cursor.fetchone()

# Print the result
print("Result:", result)

# Close the connection
conn.close()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Generated SQL Query: SELECT AVG(Employees.Salary) FROM Employees JOIN Departments ON Employees.Department = Departments.Department_Name WHERE Departments.Location = 'New York'
Result: (52500.0,)
